# 02. Explore loop inventory
## Project: Bicycle node network loop analysis

This notebook explores the loop inventory created in notebook 01.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2024-01-29  
Last modified: 2024-01-29  

## To do

## Parameters

In [ ]:
cycle_numnode_bound = 30
scenario = "scenario1"

restrictions = {"scenario1": {"cyclelengthmin": 3000, "cyclelengthmax": 15000, "maxslope": 5}, # family with small children
                "scenario2": {"cyclelengthmin": 10000, "cyclelengthmax": 40000, "maxslope": 7} # teenage/adult leisure cyclist
                }

PATH = {
    "data_in": "../data/input/faxe/network/",
    "data_out": "../data/processed/faxe/",
    "plot": "../plots/faxe/"
}

## Imports

In [ ]:
import geopandas as gpd
import igraph as ig
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from functools import reduce
import pickle 
from itertools import compress

## Functions

In [ ]:
def NormalizeData(data):
    return list((data - np.min(data)) / (np.max(data) - np.min(data)))

def getLayout(G, nodes_id, nodes_coords):
    named_vertex_list = G.vs()["name"]
    layout = []
    for n in named_vertex_list:
        pos = nodes_id.index(n)
        layout.append(nodes_coords[pos])
    return layout

def plotCheck(G, nodes_id, nodes_coords, vertex_size = 6, edge_color=None):
    fig, ax = plt.subplots()
    layout = getLayout(G, nodes_id, nodes_coords)
    if edge_color is None:
        ig.plot(G, target=ax, vertex_size=vertex_size, layout=layout);
    else:
        ig.plot(G, target=ax, vertex_size=vertex_size, layout=layout, edge_color=edge_color);
    return fig
    
def getCycleLength(c):
    l = 0
    cl = len(c)
    for i in range(cl):
        l += Gnx[c[i%cl]][c[(i+1)%cl]]["weight"]
    return l

## Exploration

### Load data

In [ ]:
with open(PATH['data_out'] + 'cycleinventory_'+str(cycle_numnode_bound)+'.pkl', 'rb') as f:
    allcycles = pickle.load(f)
    allcyclelengths = pickle.load(f)
    allcyclenumnodes = pickle.load(f)
    allcyclemaxslopes = pickle.load(f)
    G = pickle.load(f)
    cycle_numnode_bound = pickle.load(f)
    nodes_id = pickle.load(f)
    nodes_coords = pickle.load(f)
    numcycles = pickle.load(f)

In [ ]:
# Plot to double-check
plotCheck(G, nodes_id, nodes_coords);

In [ ]:
fig = plt.figure(figsize=(8, 3))
axes1 = fig.add_axes([0.1, 0.1, 0.35, 0.8])
axes2 = fig.add_axes([0.55, 0.1, 0.35, 0.8])

axes1.hist(allcyclelengths, density=True)
axes1.set_xlabel('Length [m]')
axes1.set_ylabel('Probability')
axes1.set_title('Cycle lengths')

axes2.hist(allcyclenumnodes, density=True, bins=list(range(cycle_numnode_bound+1)))
axes2.set_xlabel('Nodes')
axes2.set_title('Nodes per cycle')
axes2.set_xlim([0, cycle_numnode_bound+0.5])

plt.text(cycle_numnode_bound/20,0.01, "Bound: " + str(cycle_numnode_bound))
plt.text(cycle_numnode_bound/20,0.04, "Cycles: " + str(numcycles));

## Restrict to length bounds

Go through all nodes and restrict their cycles to the scenario's length restrictions

In [ ]:
allcycles_bound = {}
for sourcenode in allcycles:
    this_lengths = np.array(allcycles[sourcenode]['lengths'])
    mask_this = (this_lengths >= restrictions[scenario]["cyclelengthmin"]) & (this_lengths <= restrictions[scenario]["cyclelengthmax"])
    allcycles_bound[sourcenode] = {"cycles": list(compress(allcycles[sourcenode]['cycles'], mask_this)),
                                   "lengths": list(compress(allcycles[sourcenode]['lengths'], mask_this)),
                                   "numnodes": list(compress(allcycles[sourcenode]['numnodes'], mask_this))}

In [ ]:
vertex_sizes=[len(allcycles_bound[k]['cycles']) for k in range(len(allcycles_bound.keys()))]
numcycles_max = max(vertex_sizes)
vertex_sizes = [i/(numcycles_max/20) for i in vertex_sizes]

In [ ]:
fig = plotCheck(G, nodes_id, nodes_coords, vertex_size = vertex_sizes)
plt.text(0,0.04, "Max # of loops: " + str(numcycles_max));
plt.tight_layout()

In [ ]:
fig.savefig(PATH["plot"] + "network_numcycles_"+str(restrictions[scenario]["cyclelengthmin"])+"to"+str(restrictions[scenario]["cyclelengthmax"]))

## Restrict to acceptable gradients

In [ ]:
allcycles_bound = {}
for sourcenode in allcycles:
    this_lengths = np.array(allcycles[sourcenode]['lengths'])
    this_maxslopes = np.array(allcycles[sourcenode]['max_slopes'])
    mask_this = (this_lengths >= restrictions[scenario]["cyclelengthmin"])
    mask_this &= (this_lengths <= restrictions[scenario]["cyclelengthmax"]) 
    mask_this &= (this_maxslopes <= restrictions[scenario]["maxslope"])
    allcycles_bound[sourcenode] = {"cycles": list(compress(allcycles[sourcenode]['cycles'], mask_this)),
                                   "lengths": list(compress(allcycles[sourcenode]['lengths'], mask_this)),
                                   "numnodes": list(compress(allcycles[sourcenode]['numnodes'], mask_this))}

In [ ]:
vertex_sizes=[len(allcycles_bound[k]['cycles']) for k in range(len(allcycles_bound.keys()))]
numcycles_max = max(vertex_sizes)
vertex_sizes = [i/(numcycles_max/20) for i in vertex_sizes]

In [ ]:
edge_colors = []
for e in G.es:
    if e["max_slope"] > restrictions[scenario]["maxslope"]:
        edge_colors.append("red")
    else:
        edge_colors.append("grey")

In [ ]:
fig = plotCheck(G, nodes_id, nodes_coords, vertex_size=vertex_sizes, edge_color=edge_colors)
plt.text(0,0.04, "Max # of loops: " + str(numcycles_max));
plt.tight_layout()

In [ ]:
fig.savefig(PATH["plot"] + "network_numcycles_"+str(restrictions[scenario]["cyclelengthmin"])+"to"+str(restrictions[scenario]["cyclelengthmax"])+"_maxslope"+str(restrictions[scenario]["maxslope"]))